# Maps of the Iberian Peninsula

Olivier Defaux(1), Gerd Graßhoff(1,2), Mohammad Yeghaneh(2)

1: Max-Planck-Institute for the History of Science, Berlin; 
2: Humboldt University, Berlin

Date:

Ptolemy's catalogue of localities contains a list of places with their coordinates that enables to draw maps of the Iberian Peninsula. Ptolemy's text came down to us in two versions that are sometimes different regarding the spelling of the toponyms and the values of the coordinates. These versions are usually called "Omega recension" and "Xi recension". Based on the Greek text of Ptolemy's catalogue edited by A. Stückelberger and G. Graßhoff (2006), we will draw Ptolemy's map of the Iberian Peninsula and compare the two transmitted versions of his work.

In [1]:
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
%load_ext autoreload
%autoreload 2

In [2]:
from bokeh.plotting import figure, output_file, show, ColumnDataSource
from bokeh.io import output_notebook, output_file, save
from bokeh.models import HoverTool,BoxZoomTool,ResetTool, WheelZoomTool
from bokeh.layouts import row, gridplot, layout

In [3]:
pd.set_option('display.max_colwidth', -1)
pd.set_option('max_colwidth', 260)

In [4]:
options = {"compact": True, "bg": "#09a3d5",
           "color": "white", "font": "Source Sans Pro","collapse_phrases":False}

In [78]:
from tools import Geography, reformatCoord,reformatIntFrac,flatten_list, Js2Geodf

## Resources

We selected the part of Ptolemy's catalogue of localities related to the Iberian Peninsula (*Geography* 2.4–6) and we encoded the Greek text of the two recensions (Omega and Xi). The files can be loaded from the open-access plateform Zenodo.

### Import

In [6]:
# import .json file from DOI number

In [79]:
Omega = pd.read_json('../data/OmegaStructure.json',encoding="utf8")
Xi = pd.read_json('../data/XiStructure.json',encoding="utf8")

### Transform the .json files into dataframes

def findTS(s,df):
    out=""
    om=json_normalize(df,"section")
    for i,r in om.iterrows():
        sci=r["sec_ID"]
        if sci in s:
            out=r["type_sec"]
        
    return(out)

def Js2Geodf(df): 
# df=OmegaJ["chapters"][0]
    om=json_normalize(df,"section")
    om=om.dropna(subset=["sec_part"])
    l=[]
    for i,x in om.iterrows():
        k={"type_sec":x["type_sec"]}
        l.append([x["sec_part"]])
#
    listItems=list(flatten_list(l))
    dfout=pd.DataFrame(listItems)
    dfout["type_sec"]=dfout.apply(lambda x: findTS(x["ID"],df),axis=1)
    return(dfout)

The .json files reproduce Ptolemy's text and structure. In order to work with the coordinates in an efficient way, we will transform the file format to produce a simple dataframe.

In [80]:
dfOmega=Js2Geodf(Omega["chapters"][0])
dfXi=Js2Geodf(Xi["chapters"][0])

In [81]:
# Sample of the dataframe for the Omega recension:
dfOmega

,ID,category,coord,people,text,toponym,type,type_sec
0,2.04.01.01,NaN,NaN,NaN,τῆς Ἰσπανίας κατὰ δὲ Ἕλληνας Ἰβηρίας τρεῖς εἰσιν ἐπαρχίαι·,NaN,text string,area presentation
1,2.04.01.02,NaN,NaN,NaN,Βαιτικὴ καὶ Λουσιτανία καὶ Ταρρακωνησία,NaN,text string,area presentation
2,2.04.02.01,NaN,NaN,NaN,καὶ τῆς μὲν Βαιτικῆς ἡ πρὸς δυσμὰς καὶ ἄρκτους πλευρὰ ἀφορίζεται τῇ τε Λουσιτανίᾳ· καὶ τῆς Ταρρακωνησίας μέρει,NaN,text string,borders description
3,2.04.02.02,NaN,NaN,NaN,ἧς πλευρᾶς ἡ περιγραφὴ ἔχει οὕτως·,NaN,text string,borders description
4,2.04.03.01,"[river mouth, boundary]","{'long': {'integer': 'δ', 'fraction': 'ιβ'}, 'lat': {'integer': 'λζ', 'fraction': 'γο'}}",Turdetani,NaN,<τὸ δυσμικώτερον στόμα Ἄνα τοῦ ποταμοῦ>,locality,coast section
5,2.04.03.02,river mouth,"{'long': {'integer': 'δ', 'fraction': 'γ'}, 'lat': {'integer': 'λζ', 'fraction': 'L'}}",Turdetani,NaN,τὸ ἀνατολικώτερον στόμα Ἄνα τοῦ ποταμοῦ,locality,coast section
6,2.04.03.03,river path,"{'long': {'integer': 'ς', 'fraction': 'γ'}, 'lat': {'integer': 'λθ', 'fraction': ''}}",,NaN,ἡ πρὸς ἀνατολὰς τοῦ ποταμοῦ ἐπιστροφή,locality,coast section
7,2.04.03.04,"[river path, boundary]","{'long': {'integer': 'θ', 'fraction': ''}, 'lat': {'integer': 'λθ', 'fraction': ''}}",,NaN,τὸ πρὸς τῷ πέρατι τῆς Λουσιτανίας τοῦ ποταμοῦ μέρος,locality,coast section
8,2.04.03.07,boundary,"{'long': {'integer': 'ιβ', 'fraction': ''}, 'lat': {'integer': 'λζ', 'fraction': 'δ'}}",,NaN,καὶ ἡ ἐντεῦθεν γραμμὴ γραφομένη παρὰ τὴν Ταρρακωνησίαν ὡς ἐπὶ τὸ Βαλιαρικὸν πέλαγος πέρας ἔχουσα,locality,coast section
9,2.04.03.08,river source,"{'long': {'integer': 'ιδ', 'fraction': ''}, 'lat': {'integer': 'μ', 'fraction': ''}}",,NaN,αἱ δὲ πηγαὶ τοῦ ποταμοῦ ἐπέχουσι μοίρας,locality,coast section


## Transform Greek numbers to decimal coordinates

### Greek numeral notation

Each latitude and each longitude is composed of two parts: an integer part (a whole number of degrees) and a decimal part (written as a fraction of degrees).
In order to plot the localities, we have to transpose the Greek numeral system used by Ptolemy into a modern format, using decimal. The symbols used by Ptolemy to express the coordinates are Greek letters and a special sign for the fraction 1/2.

In [82]:
gfrac={"":0,"ιβ":1/12,"ς":1/6,"δ":1/4,"γ":1/3,"γιβ":5/12,"L":1/2,"Lιβ":7/12,"γο":2/3,"Lδ":3/4,"Lγ":5/6,"Lγιβ":11/12,"η":1/8,"Lς":2/3,"ςL":2/3}
gint={"":0,"α":1,"β":2,"γ":3,"δ":4,"ε":5,"ς":6,"ζ":7,"η":8,"θ":9,"ι":10,"κ":20,"λ":30,"μ":40}

In [83]:
pd.DataFrame(list(gint.items()), columns=['Greek', 'Integers']).sort_values('Integers').set_index('Greek').T

Greek,,α,β,γ,δ,ε,ς,ζ,η,θ,ι,κ,λ,μ
Integers,0,1,2,3,4,5,6,7,8,9,10,20,30,40


In [84]:
pd.DataFrame(list(gfrac.items()), columns=['Greek', 'Decimal']).sort_values('Decimal').set_index('Greek').T

Greek,,ιβ,η,ς,δ,γ,γιβ,L,Lιβ,γο,Lς,ςL,Lδ,Lγ,Lγιβ
Decimal,0.0,0.083333,0.125,0.166667,0.25,0.333333,0.416667,0.5,0.583333,0.666667,0.666667,0.666667,0.75,0.833333,0.916667


### Translate into modern numeral notation

We create two columns for each dataframe: one for the longitude, one for the latitude.

In [85]:
# Omega
dfTemp = dfOmega.copy()
dfTemp['longitude'] = dfOmega.apply(lambda row: reformatCoord(row,'long','coord'),axis=1).apply(reformatIntFrac)
dfTemp['latitude'] = dfOmega.apply(lambda row: reformatCoord(row,'lat','coord'),axis=1).apply(reformatIntFrac)

In [86]:
dfTemp[26:31]

,ID,category,coord,people,text,toponym,type,type_sec,longitude,latitude
26,2.04.06.02,city,"{'long': {'integer': 'ς', 'fraction': 'L'}, 'lat': {'integer': 'λς', 'fraction': 'L'}}",Bastuli,NaN,Μενραλία,locality,coast section,6.500000,36.500000
27,2.04.06.03,city,"{'long': {'integer': 'ς', 'fraction': 'γο'}, 'lat': {'integer': 'λς', 'fraction': 'γ'}}",Bastuli,NaN,Τρανσδούκτα,locality,coast section,6.666667,36.333333
28,2.04.06.04,city,"{'long': {'integer': 'ζ', 'fraction': 'δ'}, 'lat': {'integer': 'λς', 'fraction': 'ς'}}",Bastuli,NaN,Βαρβησόλα,locality,coast section,7.250000,36.166667
29,2.04.06.05,city,"{'long': {'integer': 'ζ', 'fraction': 'L'}, 'lat': {'integer': 'λς', 'fraction': 'ς'}}",Bastuli,NaN,Καρτμία,locality,coast section,7.500000,36.166667
30,2.04.06.06,mountain,"{'long': {'integer': 'ζ', 'fraction': 'L'}, 'lat': {'integer': 'λς', 'fraction': 'δ'}}",Bastuli,NaN,Κάρπη ὅρος καὶ στήλη τῆς ἐντὸς θαλάσσης,locality,coast section,7.500000,36.250000


In [87]:
# Xi
dfTempX = dfXi.copy()
dfTempX['longitude'] = dfXi.apply(lambda row: reformatCoord(row,'long','coord'),axis=1).apply(reformatIntFrac)
dfTempX['latitude'] = dfXi.apply(lambda row: reformatCoord(row,'lat','coord'),axis=1).apply(reformatIntFrac)

## Data preparation for the map drawing

### Coasts of the peninsula (Omega)

We select the localities that form the coasts of the peninsula and sort them in the correct order.

In [88]:
dfOmegaCoasts = dfTemp[(dfTemp.type_sec == 'coast section') & (dfTemp.type == 'locality')]
dfOmegaCoasts = dfOmegaCoasts[dfOmegaCoasts.category.apply(lambda row: row not in ['river path','river source'])]

In [89]:
dfOmegaCoasts = dfOmegaCoasts[39:70].sort_index(ascending=False).append(dfOmegaCoasts[29:38].sort_index(ascending=False)).append(dfOmegaCoasts[:2]).append(dfOmegaCoasts[4:29]).append(dfTemp[(dfTemp.ID == '2.04.08.08')]).append(dfOmegaCoasts[3:4]).append(dfOmegaCoasts[70:99])

We save the dataframe as a dictionary for later use.

In [90]:
OmegaCoasts = dfOmegaCoasts[['longitude','latitude']].values

In [91]:
OmegaCoasts;

### Inland boundary lines (Omega)

Ptolemy's map of the Iberian peninsula contains several boundary lines between the provinces that we want to draw. We save the dataframes as dictionaries for later use.

In [92]:
# boundaries of the Baetica province
boundary_om1 = dfTemp[(dfTemp.ID == '2.04.03.01')].append(dfTemp[(dfTemp.ID == '2.04.03.03')]).append(dfTemp[(dfTemp.ID == '2.04.03.04')]).append(dfTemp[(dfTemp.ID == '2.04.03.07')])
boundary_om1 = boundary_om1[['longitude','latitude']].values

#boundaries of the Lusitania province
boundary_om2 = dfTemp[(dfTemp.ID == '2.05.01.04')].append(dfTemp[(dfTemp.ID == '2.05.01.06')]).append(dfTemp[(dfTemp.ID == '2.05.04.05')]).append(dfTemp[(dfTemp.ID == '2.04.03.04')])
boundary_om2 = boundary_om2[['longitude','latitude']].values

The Pyrenees are defined by four points: three are defined by Ptolemy in the catalogue of the Iberian localities, one is defined in the chapter devoted to the province of Gallia Aquitania.

In [93]:
boundary_om3 = pd.DataFrame({'longitude':[15,17,19,20.333], 'latitude':[45.833,43,43.167,42.333]}).values

In [94]:
# make a dictionary for all coasts's lines using Omega for later use in function plot_maps
ob_dict={"OmegaCoasts":OmegaCoasts,'boundary_om1' : boundary_om1, 'boundary_om2' : boundary_om2, "boundary_om3":boundary_om3 }

### Coasts of the peninsula (Xi)

In [95]:
dfXiCoasts = dfTempX[(dfTempX.type_sec == 'coast section') & (dfTempX.type == 'locality')]
dfXiCoasts = dfXiCoasts[dfXiCoasts.category.apply(lambda row: row not in ['river path','river source'])]

In [96]:
dfXiCoasts = dfXiCoasts[37:68].sort_index(ascending=False).append(dfXiCoasts[27:36].sort_index(ascending=False)).append(dfXiCoasts[:2]).append(dfXiCoasts[4:27]).append(dfTempX[(dfTempX.ID == '2.04.08.08')]).append(dfXiCoasts[3:4]).append(dfXiCoasts[70:99])

In [97]:
# other code suggestion:
#i = dfXiCoasts.copy()
#dfXiCoasts = pd.concat([i[37:68].sort_index(ascending=False),i[27:36].sort_index(ascending=False),i[:2],i[4:27],dfTempX[(dfTempX.ID == '2.04.08.08')],i[3:4],i[70:99]]);

In [98]:
XiCoasts = dfXiCoasts[['longitude','latitude']].values

### Inland boundary lines (Xi)

In [99]:
# boundaries of the Baetica province
boundary_xi1 = dfTempX[(dfTempX.ID == '2.04.03.01')].append(dfTempX[(dfTempX.ID == '2.04.03.03')]).append(dfTempX[(dfTempX.ID == '2.04.03.04')]).append(dfTempX[(dfTempX.ID == '2.04.03.07')])

#boundaries of the Lusitania province
boundary_xi2 = dfTempX[(dfTempX.ID == '2.05.01.04')].append(dfTempX[(dfTempX.ID == '2.05.01.06')]).append(dfTempX[(dfTempX.ID == '2.05.04.05')]).append(dfTempX[(dfTempX.ID == '2.04.03.04')])

In [100]:
boundary_xi3 = pd.DataFrame({'longitude':[15.167,17,19,20.333], 'latitude':[45.833,43,43.167,42.333]}).values

In [101]:
XiCoasts = dfXiCoasts[['longitude','latitude']].values
boundary_xi1 = boundary_xi1[['longitude','latitude']].values
boundary_xi2 = boundary_xi2[['longitude','latitude']].values

In [102]:
xb_dict={'XiCoasts':XiCoasts, 'boundary_xi1' : boundary_xi1, 'boundary_xi2' : boundary_xi2, 'boundary_xi3' : boundary_xi3}

## Map of the Iberian peninsula, Omega recension

In [103]:
df_dict={"dfTemp" : dfTemp,  "dfTempX" : dfTempX}

In [104]:
fig=Geography(ob_dict,xb_dict,df_dict)

In [105]:
fig.plot_recension(ob_dict,xb_dict,df_dict,"Omega")

## Map of the Iberian peninsula, Xi recension

In [108]:
fig.plot_recension(ob_dict,xb_dict,df_dict,"Xi")

## Comparison between the two recensions

In [35]:
fig.plot_recension_all(ob_dict,xb_dict,df_dict)

In [36]:
fig.plot_compare_recension(ob_dict,xb_dict,df_dict)